In [150]:
from types import SimpleNamespace
import time
import numpy as np
import scipy as sp
from scipy import linalg
from scipy import optimize
from scipy import interpolate
import sympy as sm
import math
from math import log
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from numpy import array
from scipy import optimize,arange

%matplotlib inline
import plotly.graph_objects as go
from matplotlib.widgets import Slider, Button
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Cournot competition

- We consider a market with two firms (firm 1 and firm 2). They produce a homogenous good.
- They choose simultaneously what quantity to produce each. We note this as $q_1$ and $q_2$. The total quantity in the market is then given as $Q=q_1+q_2$.
- The market price is decreasing in quantity and is given as: $P(Q)=a-Q$.
- Both firms have the same marginal cost c, with $0 \le c<a$, and no fixed cost.
- The firms wish to maximize profits: ${\pi}_i(q_i,q_j)=P(q_i+q_j)q_i-cq_i$ 
- or: ${\pi_1}(q_1,q_2)=(a-q_1-q_2-c)q_1$ and ${\pi_2}(q_1,q_2)=(a-q_1-q_2-c)q_2$
- The solutions are best-response functions in a Nash Equilibrium where firm 1 is choosing the best response to firm 2 and vice versa.

Shown more extensively, these are the profit functions of each firm:

$$ 
π_1 (q_1 ,q_2 ) = (a - q_1 - q_2 - c)q_1 → q_1 = \left(\frac{a - q_2 -c}{2}\right)
$$
$$ 
π_2 (q_1 ,q_2 ) = (a - q_1 - q_2 - c)q_2 →  q_2 = \left(\frac{a - q_1 -c}{2}\right)
$$

By taking the first-order derivative with respect to $q_1$ and $q_2$, we obtain the following equations:

$$ 
a - 2q_1 - q_2 - c = 0
$$
$$
a - 2q_2 - q_1 - c = 0
$$

and the Best-Response functions for each firm. In the equlibrium we illustrate the best-response quantities with a *.

$$ q_1^* = \left(\frac{a - q_2^*}{2}\right) $$

$$ q_2^* = \left(\frac{a - q_1^*}{2}\right) $$

In the Nash equlibrium, the optimal output level of both firms is given by:

$$ q_1^* = q_2^* = \left(\frac{a - c}{3}\right) $$

In the equlibrium, the optimal quantity, price and profits are the following:

$$ Q = 2 \left(\frac{a - c}{3}\right) $$

$$ P = a - 2 \left(\frac{a - c}{3}\right) $$

$$ π_1 (q_1^* ,q_2^* ) = P(Q^*)q_1^* - cq_1^* = \left(\frac{a + 2c}{3}\right) - c\left(\frac{a - c}{3}\right) $$

$$ = \left(\frac{a - c}{3}\right)^2 $$

In [151]:
#defining variables
Q=sm.symbols("Q")
q1=sm.symbols("q_1") 
q2=sm.symbols("q_2")
q1_star=sm.symbols("q_1^*") 
q2_star=sm.symbols("q_2^*")
qi=sm.symbols("q_i") 
qj=sm.symbols("q_j")
P=sm.symbols("P")
pi_1=sm.symbols("\pi_1")
pi_2=sm.symbols("\pi_2")

#defining paraameters
a=sm.symbols("a")
c=sm.symbols("c")


# Analytical solution

To find the analytical solution, we find the derivative for each profit function as the firms wish to maximize profits.

Solve in **five steps**:

1. **Take the derivative** wrt. $q_1$
2. **Isolate** $q_2$ from the foc for firm 1
3. **Substitute** in $q_2$ for firm 2
4. **Solve the FOC** for $q_1$
5. **substitute** to find $q_2$

In [152]:
#defining functions
profit_1=(a-q1-q2-c)*q1
profit_2=(a-q1-q2-c)*q2

In [153]:
#take derivatives
foc_1 = sm.diff(profit_1,q1)
foc_2 = sm.diff(profit_2,q2)
foc_1,foc_2

(a - c - 2*q_1 - q_2, a - c - q_1 - 2*q_2)

In [154]:
#isolate q_1 in foc_1
q1_from_profit_1 = sm.solve(sm.Eq(foc_1,0), q1)
q1_from_profit_1[0]

a/2 - c/2 - q_2/2

In [155]:
#substitute in to foc_2 
foc_2_subs=foc_2.subs(q1,q1_from_profit_1[0])
sm.Eq(q1_star,foc_2_subs)

Eq(q_1^*, a/2 - c/2 - 3*q_2/2)

In [156]:
#isolate q2
q_2_foc_2_subs=sm.solve(sm.Eq(foc_2_subs,0), q2)
sm.Eq(q2_star,q_2_foc_2_subs[0])

Eq(q_2^*, a/3 - c/3)

As the firms produce a homogenous good, we get the following solution:
$q_1^*=q_2^*=(a-c)/3$

# Numerical solution

In [157]:
#defining functions
#market price as a function of quantity
def p(q1,q2,a):
    p = a - q1 - q2
    return p

def Q(q1,q2):
    Q=q1+q2
    return Q

def cost(q,c):
    cost = q*c
    return cost

#profits for firm 1
def pi_1(q1,q2,a,c):
    profits_1 = (p(q1,q2,a)-c)*q1
    return profits_1

#profits for firm 2
def pi_2(q1,q2,a,c):
    profits_2 = (p(q1,q2,a)-c)*q2
    return profits_2

In [158]:
#Lowest value
q0 = [0]

#best responses 
def bestresp_1(q2,a,c):
    q1_star_num = optimize.minimize(lambda q0: -pi_1(q0,q2,a,c), q0).x[0]
    return q1_star_num

def bestresp_2(q1,a,c):
    q2_star_num = optimize.minimize(lambda q0: -pi_2(q1,q0,a,c), q0).x[0]
    return q2_star_num

In [159]:
#defining nash equilibrium
def nash_eq(q,param):
    x = q[0] - bestresp_1(q[1],param[0],param[1])
    y = q[1] - bestresp_2(q[0],param[0],param[1])
    return [x,y]

In [160]:
#alt. definition (http://janboone.github.io/competition_policy_and_regulation/Collusion_Cournot/Collusion_Cournot.html?fbclid=IwAR0r6GQS8oBTqLwuc1berXM_yU14l-x4XuVoI1I2zeoN4j1ZNryrviFKhuM)
def nash_eq(q,param): # vector param = (b,c1,c2)
    return np.array(q)-np.array([bestresp_1(q[1],param[0],param[1]),bestresp_2(q[0],param[0],param[1])])

In [161]:
inital_values = [3,2]
param = [3,2]

In [162]:
solver = optimize.fsolve(nash_eq,inital_values, args = (param))
print(f'The Nash Equilibrium is at: {solver}')

The Nash Equilibrium is at: [0.33333333 0.33333333]


# Plotting the results in interactive graphs

# Model extension: Cournot model with three firms

In [163]:
# Define the functions for the three-firms case
def p(q1,q2,q3,a):
    p = a - q1 - q2 - q3
    return p

def Q(q1,q2,q3):
    Q=q1+q2+q3
    return Q

def cost(q,c):
    cost = q*c
    return cost

#profits for firm 1
def pi_1(q1,q2,q3,a,c):
    profits_1 = (p(q1,q2,q3,a)-c)*q1
    return profits_1

#profits for firm 2
def pi_2(q1,q2,q3,a,c):
    profits_2 = (p(q1,q2,q3,a)-c)*q2
    return profits_2

#profits for firm 3
def pi_3(q1,q2,q3,a,c):
    profits_3 = (p(q1,q2,q3,a)-c)*q2
    return profits_3

In [164]:
#Lowest value
q0 = [0]

#best responses 
def bestresp_1(q2,q3,a,c):
    q1_star_num = optimize.minimize(lambda q0: -pi_1(q0,q2,q3,a,c), q0).x[0]
    return q1_star_num

def bestresp_2(q1,q3,a,c):
    q2_star_num = optimize.minimize(lambda q0: -pi_2(q1,q0,q3,a,c), q0).x[0]
    return q2_star_num

def bestresp_3(q1,q2,a,c):
    q3_star_num = optimize.minimize(lambda q0: -pi_3(q1,q2,q0,a,c), q0).x[0]
    return q3_star_num

In [165]:
#defining nash equilibrium
def nash_eq_3f(q,param):
    x = q[0] - bestresp_1(q[1],q[2],param[0],param[1])
    y = q[1] - bestresp_2(q[0],q[2],param[0],param[1])
    z = q[2] - bestresp_3(q[0],q[1],param[0],param[1])
    return [x,y,z]

In [166]:
inital_values = [1,1,1]
param = [11,2]

In [167]:
solver = optimize.fsolve(nash_eq_3f,inital_values, args = (param))
print(f'The Nash Equilibrium is at: {solver}')

C:\Users\heta3\.anaconda_empty\lib\site-packages\scipy\optimize\optimize.py:1276: RuntimeWarning: overflow encountered in multiply
  Hk = np.dot(A1, np.dot(Hk, A2)) + (rhok * sk[:, np.newaxis] *
C:\Users\heta3\.anaconda_empty\lib\site-packages\scipy\optimize\linesearch.py:287: RuntimeWarning: invalid value encountered in multiply
  return f(xk + alpha * pk, *args)
C:\Users\heta3\.anaconda_empty\lib\site-packages\scipy\optimize\optimize.py:1276: RuntimeWarning: overflow encountered in multiply
  Hk = np.dot(A1, np.dot(Hk, A2)) + (rhok * sk[:, np.newaxis] *
C:\Users\heta3\.anaconda_empty\lib\site-packages\scipy\optimize\linesearch.py:287: RuntimeWarning: invalid value encountered in multiply
  return f(xk + alpha * pk, *args)
C:\Users\heta3\.anaconda_empty\lib\site-packages\scipy\optimize\optimize.py:1276: RuntimeWarning: overflow encountered in multiply
  Hk = np.dot(A1, np.dot(Hk, A2)) + (rhok * sk[:, np.newaxis] *
C:\Users\heta3\.anaconda_empty\lib\site-packages\scipy\optimize\linesea

The Nash Equilibrium is at: [2.23420491 1.00000001 1.        ]


# Conclusion